# BioNeMo Fine-Tuning & Inference

> Inspired by
> - [BioNeMo2 Inference Notebook](https://github.com/NVIDIA/bionemo-framework/blob/v2.3/docs/docs/user-guide/examples/bionemo-esm2/inference.ipynb)
> - [BioNeMo2 Inference Example](https://docs.nvidia.com/bionemo-framework/latest/user-guide/examples/bionemo-esm2/inference/)

Before running fine-tuning, take a look at `02_bionemo_fine-tune.py` to see how fine-tuning runs are set up in BioNeMo. Some interesting sections to look at are:

- nl.MegatronStrategy - How model parallelism is configured
- biobert_lightning_module - Using PEFT for training
- ESM2FineTuneSeqConfig - Config for sequence classification
- nl.Trainer - Set the number of GPUs and nodes

After going through the code, run it with the cell below

In [ ]:
# Run fine-tuning
! python 02_bionemo_fine-tune.py

At the end of the run, you should see a `Experiment completed with checkpoint stored at` message pointing to final checkpoint. Update `checkpoint_path` with the name of your model. Feel free to update `work_dir` if `/tmp` is limited. `valid_csv` should also have already been created in `01_hf_fine-tune.ipynb`.

In [ ]:
# Update this checkpoint after fine-tuning
checkpoint_path = "/tmp/tmpxs8k_u4j/finetune_regressor/checkpoints/finetune_regressor--reduced_train_loss=0.0302-epoch=0-consumed_samples=1600.0-last"

# This gets created after running 01_hf_fine-tune.ipynb
valid_csv = "/tmp/test_df.csv"
work_dir = "/tmp/work"

After fine-tuning a model, it's good to see how well it performs. This can be done by running inference on all the test data we previously cached in `/tmp/test_df.csv`.

Similar to fine-tuning, BioNeMo inference mush be run in a separate process, which is easy to do through calling a shell with `!` in Jupyter.

| Argument | Description |
|:========:|:============|
| checkpoint-path | path to model checkpoint |
| data-path | CSV with "sequences" and "labels" columns |
| results-path | Where output `.pt` will be written |
| micro-batch-size | Number of inputs in each GPU batch |
| num-gpus | Number of GPUs to use for inference |
| precision | Model precision to use |
| config-class | ESM2FineTuneSeqConfig tells BioNeMo that we did sequence classification |

In [ ]:
! infer_esm2 --checkpoint-path {checkpoint_path} \
             --data-path {valid_csv} \
             --results-path {work_dir} \
             --micro-batch-size 3 \
             --num-gpus 1 \
             --precision "bf16-mixed" \
             --include-hiddens \
             --include-embeddings \
             --include-logits \
             --include-input-ids \
             --config-class ESM2FineTuneSeqConfig

Since BioNeMo is run from a separate process, output is written to a `.pt` file. This can be loaded using `torch.load` and the contents can be examined.

In [ ]:
import torch
results = torch.load(f"{work_dir}/predictions__rank_0.pt")

# Print out the contents of inference results
for key, val in results.items():
    if val is not None:
        print(f'{key}\t{val.shape}')

The `regression_output` key contains all the classifications. To understand how well the model did at fine-tuning, we can append it to the test dataframe after rounding the classification to 0 or 1.

In [ ]:
import pandas

# Print out the original sequence, true label, and inferred label
test_df = pandas.read_csv("/tmp/test_df.csv")[['sequences','labels']]
test_df['inference'] = results['regression_output'].round().int().numpy()
test_df

Accuracy or other metrics and then be computed from these results. How did your model do?

In [ ]:
# Check accuracy from whole dataset
from sklearn.metrics import accuracy_score
accuracy_score(test_df['labels'], test_df['inference'])